In [36]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input , Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [20]:
data = pd.read_csv("IMDB Dataset.csv")

In [21]:
data.shape

(50000, 2)

In [22]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [23]:
data["sentiment"].value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

In [24]:
data.replace({"sentiment": {"positive": 1, "negative": 0}}, inplace=True)

In [25]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [26]:
# split data into training data and test data
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

In [27]:
print(train_data.shape)
print(test_data.shape)

(40000, 2)
(10000, 2)


In [28]:
# Tokenize text data
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_data["review"])
X_train = pad_sequences(tokenizer.texts_to_sequences(train_data["review"]), maxlen=200)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data["review"]), maxlen=200)

In [29]:
print(X_train)

[[1935    1 1200 ...  205  351 3856]
 [   3 1651  595 ...   89  103    9]
 [   0    0    0 ...    2  710   62]
 ...
 [   0    0    0 ... 1641    2  603]
 [   0    0    0 ...  245  103  125]
 [   0    0    0 ...   70   73 2062]]


In [30]:
Y_train = train_data["sentiment"]
Y_test = test_data["sentiment"]

In [31]:
print(Y_train)

39087    0
30893    0
45278    1
16398    0
13653    0
        ..
11284    1
44732    1
38158    0
860      1
15795    1
Name: sentiment, Length: 40000, dtype: int64


In [38]:
# build the model
input_layer = Input(shape=(200,))  # Specify the sequence length here
from tensorflow.keras.models import Model
# Create the embedding layer
embedding_layer = Embedding(input_dim=5000, output_dim=128)(input_layer)

# Add LSTM layer
lstm_layer = LSTM(128, dropout=0.2, recurrent_dropout=0.2)(embedding_layer)

# Add Dense layer
output_layer = Dense(1, activation="sigmoid")(lstm_layer)

# Build the model
model = Model(inputs=input_layer, outputs=output_layer)

In [39]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 200)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding_4 (Embedding)              │ (None, 200, 128)            │         640,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_4 (LSTM)                        │ (None, 128)                 │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 771,713 (2.94 MB)

 Trainable params: 771,713 (2.94 MB)

 Non-trainable params: 0 (0.00 B)

In [40]:
# compile the model
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [41]:
model.fit(X_train, Y_train, epochs=5, batch_size=64, validation_split=0.2)

Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 130s 255ms/step - accuracy: 0.7309 - loss: 0.5285 - val_accuracy: 0.8509 - val_loss: 0.3549
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 120s 241ms/step - accuracy: 0.8531 - loss: 0.3511 - val_accuracy: 0.8574 - val_loss: 0.3387
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 114s 228ms/step - accuracy: 0.8323 - loss: 0.3760 - val_accuracy: 0.8240 - val_loss: 0.3951
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 124s 249ms/step - accuracy: 0.8725 - loss: 0.3083 - val_accuracy: 0.8635 - val_loss: 0.3276
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 118s 236ms/step - accuracy: 0.8904 - loss: 0.2694 - val_accuracy: 0.8526 - val_loss: 0.3718


In [ ]:
loss, accuracy = model.evaluate(X_test, Y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

In [ ]:
def predict_sentiment(review):
  # tokenize and pad the review
  sequence = tokenizer.texts_to_sequences([review])
  padded_sequence = pad_sequences(sequence, maxlen=200)
  prediction = model.predict(padded_sequence)
  sentiment = "positive" if prediction[0][0] > 0.5 else "negative"
  return sentiment

In [ ]:
# example usage
new_review = "This movie was fantastic. I loved it."
sentiment = predict_sentiment(new_review)
print(f"The sentiment of the review is: {sentiment}")